In [75]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

#To install plotpy library:
#pip install plotly --upgrade

In [49]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [50]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [51]:
nodes_hl1=1000
nodes_hl2=1000
n_classes=10
batch_size=100

In [52]:
x=tf.placeholder('float',[None,784])
y=tf.placeholder('float')

In [53]:
def neural_network(data):
    hl1={'weights':tf.Variable(tf.random_normal([784,nodes_hl1])),'bias':tf.Variable(tf.random_normal([nodes_hl1]))}
    hl2={'weights':tf.Variable(tf.random_normal([nodes_hl1,nodes_hl2])),'bias':tf.Variable(tf.random_normal([nodes_hl2]))}
    output_layer={'weights':tf.Variable(tf.random_normal([nodes_hl2,n_classes])),'bias':tf.Variable(tf.random_normal([n_classes]))}
    #(input_data*weights +bias)
    #using activation function as ReLU
    l1=tf.add(tf.matmul(data,hl1['weights']),hl1['bias'])
    l1=tf.nn.relu(l1)
                                                                                        
    l2=tf.add(tf.matmul(l1,hl2['weights']),hl2['bias'])
    l2=tf.nn.relu(l2)
                                                                                           
    output=tf.add(tf.matmul(l2,output_layer['weights']),output_layer['bias'])
                                                                                           
    return output

In [167]:
summary_dir = '/tmp/tensorflow/mnist'
mnist.train.num_examples

55000

In [181]:
num_epoch=30
f=2
epoch_loss=np.zeros(num_epoch)
test_acc=[]

In [182]:
def train_neural_network(x):
    prediction=neural_network(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    #tf.summary.scalar('cost',cost)
   
    with tf.Session() as sess:
        #train_writer = tf.summary.FileWriter(summary_dir + '/train',sess.graph)
        #test_writer = tf.summary.FileWriter(summary_dir + '/test',sess.graph)
        #merged = tf.summary.merge_all()
        sess.run(tf.global_variables_initializer())
        #merged=tf.summary.merge_all()
        
        for epoch in range(num_epoch):
           
            for i in range(int(mnist.train.num_examples/batch_size)):
                batch_x,batch_y=mnist.train.next_batch(batch_size)
                _, c=sess.run([optimizer, cost],feed_dict={x:batch_x,y:batch_y})
                epoch_loss[epoch]+=c
            
                #train_writer.add_summary(summary,i,epoch)
                if i%int((int(mnist.train.num_examples/batch_size)/f))==0:
                    correct=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
                    accuracy=tf.reduce_mean(tf.cast(correct,'float'))
                    test_acc.append(accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
            
            print ('Epoch', epoch+1,'completed out of',num_epoch,'loss:',epoch_loss[epoch])
            print (time.time() - start_time, 'sec')
                
#         correct=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#         accuracy=tf.reduce_mean(tf.cast(correct,'float'))
#         tf.summary.scalar('accuracy',accuracy)
#         print('accuracy :',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
        
    

In [183]:
import time
if __name__=="__main__":
    start_time=time.time()
    train_neural_network(x)
    print ("Training Over. \nNow you are a SuperSaiyan\n")

Epoch 1 completed out of 30 loss: 202373.6941576004
13.82559609413147 sec
Epoch 2 completed out of 30 loss: 52240.040556902066
22.884214162826538 sec
Epoch 3 completed out of 30 loss: 28737.23848864436
32.06090712547302 sec
Epoch 4 completed out of 30 loss: 16293.072530255304
41.46075105667114 sec
Epoch 5 completed out of 30 loss: 9959.414911266273
51.05084705352783 sec
Epoch 6 completed out of 30 loss: 6470.603584134855
61.6993989944458 sec
Epoch 7 completed out of 30 loss: 4935.773208344019
72.29271006584167 sec
Epoch 8 completed out of 30 loss: 3780.830595511128
82.73738598823547 sec
Epoch 9 completed out of 30 loss: 3262.509975385874
92.96575713157654 sec
Epoch 10 completed out of 30 loss: 3022.774283845656
103.29224300384521 sec
Epoch 11 completed out of 30 loss: 3451.7840214311186
113.64941906929016 sec
Epoch 12 completed out of 30 loss: 3197.096623227212
123.59376907348633 sec
Epoch 13 completed out of 30 loss: 2360.844452847418
134.13972997665405 sec
Epoch 14 completed out of 3

In [95]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print (__version__)

3.1.1


In [97]:
import plotly
plotly.tools.set_credentials_file(username='ymehta95', api_key='TVHPG5E24omQeUyWougP')

In [184]:
x_axis1=[]
x_axis2=[]
for i in range(num_epoch):
    x_axis1.append(i+1)
    
for i in range(num_epoch*f):
    x_axis2.append((i)/f)


In [185]:
trace0=go.Scatter(x=x_axis1,y=epoch_loss)
trace1=go.Scatter(x=x_axis2,y=test_acc)
data=([trace0])
py.iplot(data,filename='mnist_adam_loss')

In [186]:
data=([trace1])
py.iplot(data,filename='mnist_adam_acc')